In [44]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [45]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [46]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [47]:
# Create path to sqlite
database_path = "../Resources/hawaii.sqlite"
# Create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///{database_path}")

In [48]:
# Reflect an existing database into a new model
Base = automap_base()

# Reflect the database tables
Base.prepare(engine, reflect=True)

In [49]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [50]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [51]:
# Create our session (link) from Python to the DB
session = Session(engine)


# Exploratory Precipitation Analysis

In [52]:
# Find the most recent date in the data set
session.query(measurement.date).order_by(measurement.date.desc()).first()

('2017-08-23',)

In [53]:
import datetime as dt

In [54]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
session.query(measurement.date,measurement.prcp).filter(measurement.date >= '2016-08-23').order_by(measurement.date).all()

# Calculate the date one year from the last date in data set.
one_year = dt.date(2017, 8, 23) + dt.timedelta(days=365)
print("One year from last date in data set: ", one_year)

One year from last date in data set:  2018-08-23


In [55]:
# Perform a query to retrieve the date and precipitation scores
results = session.query(measurement.date, measurement.prcp).filter
results

<bound method Query.filter of <sqlalchemy.orm.query.Query object at 0x0000020D2A4BDEB0>>

In [58]:
# Save the query results as a Pandas DataFrame and set the index to the date column
# query_df = pd.read_sql('measurement', f'sqlite:///{database_path}').set_index("date", inplace=True)
query_df = pd.DataFrame(results, columns = ["date","prcp"])
query_df.set_index('Date', inplace = True)
query_df = query_df.dropna()
query_df.head(20)

ValueError: DataFrame constructor not properly called!

In [ ]:
# Sort the dataframe by date
sel = [measurement.date]
all_dates = session.query(*sel).\
    group_by(measurement.date).\
        order_by(measurement.date).all()
all_dates


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
# ######Day 3 Activity 3 has a specific example'ArithmeticError
query_df.plot(figsize=(10,6))
plt.xlabel("Date")
plt.ylabel("Inches")
plt.xticks(rotation=90)
plt.ylim(0,max(query_df["Precipitation"])+1)
plt.tight_layout()
plt.savefig("../Images/preciptation_past_year.png")
plt.show()


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
###### Pandas Day 1 Activity 4
summary = query_df["prcp"].describe()
summary

# Exploratory Station Analysis

In [59]:
# Design a query to calculate the total number stations in the dataset
#####  You can use session.query() or just plain SQL for this.
total_stations = session.query(station).\
    filter(station.id).count()
print(total_stations)

9


In [60]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
count_per_station = session.query(measurement.station,func.count(measurement.station)).\
        group_by(measurement.station).\
            order_by(func.count(measurement.station).desc()).all()
count_per_station

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [64]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temp_description = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
    filter(measurement.station == 'USC00519281').all()
# temp_description
# ***********FINISH***************

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
#####  You can use session.query() or just plain SQL for this.
#####  The plot is similar to the one above


# Close session

In [ ]:
# Close Session
session.close()